# importing libraries

In [77]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
import plotly_express as px
import warnings
warnings.filterwarnings('ignore')


In [78]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix
from sklearn import metrics

### reading the dataset

In [79]:
df = pd.read_csv('hypothyroid.csv')

In [80]:
df

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH measured,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
0,41,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.3,t,2.5,t,125,t,1.14,t,109,f,?,SVHC,P
1,23,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,4.1,t,2,t,102,f,?,f,?,f,?,other,P
2,46,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.98,f,?,t,109,t,0.91,t,120,f,?,other,P
3,70,F,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.16,t,1.9,t,175,f,?,f,?,f,?,other,P
4,70,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.72,t,1.2,t,61,t,0.87,t,70,f,?,SVI,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3767,30,F,f,f,f,f,f,f,f,f,f,f,f,t,f,f,f,?,f,?,f,?,f,?,f,?,f,?,other,P
3768,68,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1,t,2.1,t,124,t,1.08,t,114,f,?,SVI,P
3769,74,F,f,f,f,f,f,f,f,f,t,f,f,f,f,f,t,5.1,t,1.8,t,112,t,1.07,t,105,f,?,other,P
3770,72,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.7,t,2,t,82,t,0.94,t,87,f,?,SVI,P


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   age                        3772 non-null   object
 1   sex                        3772 non-null   object
 2   on thyroxine               3772 non-null   object
 3   query on thyroxine         3772 non-null   object
 4   on antithyroid medication  3772 non-null   object
 5   sick                       3772 non-null   object
 6   pregnant                   3772 non-null   object
 7   thyroid surgery            3772 non-null   object
 8   I131 treatment             3772 non-null   object
 9   query hypothyroid          3772 non-null   object
 10  query hyperthyroid         3772 non-null   object
 11  lithium                    3772 non-null   object
 12  goitre                     3772 non-null   object
 13  tumor                      3772 non-null   object
 14  hypopitu

In [82]:
df.describe(include=object)

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH measured,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
count,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772,3772
unique,94,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,288,2,70,2,242,2,147,2,235,1,1,5,2
top,59,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,?,t,?,t,?,t,?,t,?,f,?,other,P
freq,95,2480,3308,3722,3729,3625,3719,3719,3713,3538,3535,3754,3738,3676,3771,3588,3403,369,3003,769,3541,231,3385,387,3387,385,3772,3772,2201,3481


In [83]:
df.head()

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH measured,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
0,41,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.3,t,2.5,t,125,t,1.14,t,109,f,?,SVHC,P
1,23,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,4.1,t,2,t,102,f,?,f,?,f,?,other,P
2,46,M,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.98,f,?,t,109,t,0.91,t,120,f,?,other,P
3,70,F,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.16,t,1.9,t,175,f,?,f,?,f,?,other,P
4,70,F,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.72,t,1.2,t,61,t,0.87,t,70,f,?,SVI,P


In [84]:
df['binaryClass'].nunique()


2

In [85]:
df['binaryClass'].value_counts()

P    3481
N     291
Name: binaryClass, dtype: int64

#### checking for null values

In [86]:
df.isnull().sum()

age                          0
sex                          0
on thyroxine                 0
query on thyroxine           0
on antithyroid medication    0
sick                         0
pregnant                     0
thyroid surgery              0
I131 treatment               0
query hypothyroid            0
query hyperthyroid           0
lithium                      0
goitre                       0
tumor                        0
hypopituitary                0
psych                        0
TSH measured                 0
TSH                          0
T3 measured                  0
T3                           0
TT4 measured                 0
TT4                          0
T4U measured                 0
T4U                          0
FTI measured                 0
FTI                          0
TBG measured                 0
TBG                          0
referral source              0
binaryClass                  0
dtype: int64

In [87]:
df.columns.tolist()

['age',
 'sex',
 'on thyroxine',
 'query on thyroxine',
 'on antithyroid medication',
 'sick',
 'pregnant',
 'thyroid surgery',
 'I131 treatment',
 'query hypothyroid',
 'query hyperthyroid',
 'lithium',
 'goitre',
 'tumor',
 'hypopituitary',
 'psych',
 'TSH measured',
 'TSH',
 'T3 measured',
 'T3',
 'TT4 measured',
 'TT4',
 'T4U measured',
 'T4U',
 'FTI measured',
 'FTI',
 'TBG measured',
 'TBG',
 'referral source',
 'binaryClass']

# feature Engineering

In [88]:
from sklearn.preprocessing import LabelEncoder

In [89]:
le = LabelEncoder()

In [90]:
df['sex'] = le.fit_transform(df['sex'])

In [91]:
a = df['age'] == '?'
df[a]

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH measured,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
1985,?,1,t,f,f,f,f,f,t,f,f,f,f,f,f,f,t,0.6,t,1.5,t,120,t,0.82,t,146,f,?,other,P


In [92]:
df.replace('?',np.nan,inplace=True) 

In [93]:
df[df['age'] == 'NaN'] 


,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH measured,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass


In [94]:
mean_nl = df['age'].astype('float').mean()
df['age'].replace(np.nan,mean_nl,inplace=True) # np.nan means empty sapce
df

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH measured,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
0,41,1,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.3,t,2.5,t,125,t,1.14,t,109,f,NaN,SVHC,P
1,23,1,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,4.1,t,2,t,102,f,NaN,f,NaN,f,NaN,other,P
2,46,2,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.98,f,NaN,t,109,t,0.91,t,120,f,NaN,other,P
3,70,1,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.16,t,1.9,t,175,f,NaN,f,NaN,f,NaN,other,P
4,70,1,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.72,t,1.2,t,61,t,0.87,t,70,f,NaN,SVI,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3767,30,1,f,f,f,f,f,f,f,f,f,f,f,t,f,f,f,NaN,f,NaN,f,NaN,f,NaN,f,NaN,f,NaN,other,P
3768,68,1,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1,t,2.1,t,124,t,1.08,t,114,f,NaN,SVI,P
3769,74,1,f,f,f,f,f,f,f,f,t,f,f,f,f,f,t,5.1,t,1.8,t,112,t,1.07,t,105,f,NaN,other,P
3770,72,2,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.7,t,2,t,82,t,0.94,t,87,f,NaN,SVI,P


In [95]:
pd.set_option('display.max_columns',None)

In [96]:
df['TBG'].nunique()

0

In [97]:
df.isnull().sum()

age                             0
sex                             0
on thyroxine                    0
query on thyroxine              0
on antithyroid medication       0
sick                            0
pregnant                        0
thyroid surgery                 0
I131 treatment                  0
query hypothyroid               0
query hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                   0
psych                           0
TSH measured                    0
TSH                           369
T3 measured                     0
T3                            769
TT4 measured                    0
TT4                           231
T4U measured                    0
T4U                           387
FTI measured                    0
FTI                           385
TBG measured                    0
TBG                          3772
referral source                 0
binaryClass   

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        3772 non-null   object 
 1   sex                        3772 non-null   int32  
 2   on thyroxine               3772 non-null   object 
 3   query on thyroxine         3772 non-null   object 
 4   on antithyroid medication  3772 non-null   object 
 5   sick                       3772 non-null   object 
 6   pregnant                   3772 non-null   object 
 7   thyroid surgery            3772 non-null   object 
 8   I131 treatment             3772 non-null   object 
 9   query hypothyroid          3772 non-null   object 
 10  query hyperthyroid         3772 non-null   object 
 11  lithium                    3772 non-null   object 
 12  goitre                     3772 non-null   object 
 13  tumor                      3772 non-null   objec

In [99]:
mean_tsh = df['TSH'].astype('float').mean()
df['TSH'].replace(np.nan,mean_tsh,inplace=True)

In [100]:
mean_t3 = df['T3'].astype('float').mean()
df['T3'].replace(np.nan,mean_t3,inplace=True)


In [101]:
mean_tt4 = df['TT4'].astype('float').mean()
df['TT4'].replace(np.nan,mean_tt4,inplace=True)


In [102]:
mean_t4U = df['T4U'].astype('float').mean()
df['T4U'].replace(np.nan,mean_t4U,inplace=True)


In [103]:
mean_fti = df['FTI'].astype('float').mean()
df['FTI'].replace(np.nan,mean_fti,inplace=True)


In [104]:
df.isnull().sum()

age                             0
sex                             0
on thyroxine                    0
query on thyroxine              0
on antithyroid medication       0
sick                            0
pregnant                        0
thyroid surgery                 0
I131 treatment                  0
query hypothyroid               0
query hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                   0
psych                           0
TSH measured                    0
TSH                             0
T3 measured                     0
T3                              0
TT4 measured                    0
TT4                             0
T4U measured                    0
T4U                             0
FTI measured                    0
FTI                             0
TBG measured                    0
TBG                          3772
referral source                 0
binaryClass   

In [105]:
df

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH measured,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
0,41,1,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1.3,t,2.5,t,125,t,1.14,t,109,f,NaN,SVHC,P
1,23,1,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,4.1,t,2,t,102,f,0.995,f,110.469649,f,NaN,other,P
2,46,2,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.98,f,2.0135,t,109,t,0.91,t,120,f,NaN,other,P
3,70,1,t,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.16,t,1.9,t,175,f,0.995,f,110.469649,f,NaN,other,P
4,70,1,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.72,t,1.2,t,61,t,0.87,t,70,f,NaN,SVI,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3767,30,1,f,f,f,f,f,f,f,f,f,f,f,t,f,f,f,5.086766,f,2.0135,f,108.319345,f,0.995,f,110.469649,f,NaN,other,P
3768,68,1,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,1,t,2.1,t,124,t,1.08,t,114,f,NaN,SVI,P
3769,74,1,f,f,f,f,f,f,f,f,t,f,f,f,f,f,t,5.1,t,1.8,t,112,t,1.07,t,105,f,NaN,other,P
3770,72,2,f,f,f,f,f,f,f,f,f,f,f,f,f,f,t,0.7,t,2,t,82,t,0.94,t,87,f,NaN,SVI,P


In [106]:
df['on thyroxine'] = le.fit_transform(df['on thyroxine'])


In [107]:
df['query on thyroxine'] = le.fit_transform(df['query on thyroxine'])

In [108]:
df['on antithyroid medication'] = le.fit_transform(df['on antithyroid medication'])

In [109]:
df['sick'] = le.fit_transform(df['sick'])


In [110]:
df['pregnant'] = le.fit_transform(df['pregnant'])


In [111]:
df['thyroid surgery'] = le.fit_transform(df['thyroid surgery'])


In [112]:
df['I131 treatment'] = le.fit_transform(df['I131 treatment'])


In [113]:
df['query hyperthyroid'] = le.fit_transform(df['query hyperthyroid'])


In [114]:
df['lithium'] = le.fit_transform(df['lithium'])


In [115]:
df['goitre'] = le.fit_transform(df['goitre'])


In [116]:
df['tumor'] = le.fit_transform(df['tumor'])


In [117]:
df['hypopituitary'] = le.fit_transform(df['hypopituitary'])


In [118]:
df['psych'] = le.fit_transform(df['psych'])


In [119]:
df['TSH measured'] = le.fit_transform(df['TSH measured'])


In [120]:
df['query hypothyroid'] = le.fit_transform(df['query hypothyroid'])


In [121]:
df['T3 measured'] = le.fit_transform(df['T3 measured'])


In [122]:
df['TT4 measured'] = le.fit_transform(df['TT4 measured'])


In [123]:
df['T4U measured'] = le.fit_transform(df['T4U measured'])


In [124]:
df['FTI measured'] = le.fit_transform(df['FTI measured'])


In [125]:
df['TBG measured'] = le.fit_transform(df['TBG measured'])


In [126]:
df['TBG'] = le.fit_transform(df['TBG'])


In [127]:
df['referral source'] = le.fit_transform(df['referral source'])


In [128]:
df['binaryClass'] = le.fit_transform(df['binaryClass'])


In [129]:
df.head()

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH measured,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,TBG measured,TBG,referral source,binaryClass
0,41,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.3,1,2.5,1,125,1,1.14,1,109,0,0,1,1
1,23,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4.1,1,2,1,102,0,0.995,0,110.469649,0,0,4,1
2,46,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.98,0,2.0135,1,109,1,0.91,1,120,0,0,4,1
3,70,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.16,1,1.9,1,175,0,0.995,0,110.469649,0,0,4,1
4,70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.72,1,1.2,1,61,1,0.87,1,70,0,0,3,1


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   age                        3772 non-null   object
 1   sex                        3772 non-null   int32 
 2   on thyroxine               3772 non-null   int32 
 3   query on thyroxine         3772 non-null   int32 
 4   on antithyroid medication  3772 non-null   int32 
 5   sick                       3772 non-null   int32 
 6   pregnant                   3772 non-null   int32 
 7   thyroid surgery            3772 non-null   int32 
 8   I131 treatment             3772 non-null   int32 
 9   query hypothyroid          3772 non-null   int32 
 10  query hyperthyroid         3772 non-null   int32 
 11  lithium                    3772 non-null   int32 
 12  goitre                     3772 non-null   int32 
 13  tumor                      3772 non-null   int32 
 14  hypopitu

In [131]:
df['age']= df['age'].astype(int)

In [132]:
df['TSH'] = df['TSH'].astype(float)

In [133]:
df['TT4'] = df['TT4'].astype(float)


In [134]:
df['FTI'] = df['FTI'].astype(float)


In [135]:
df['T4U'] = df['T4U'].astype(float)


In [136]:
df['T3'] = df['T3'].astype(float)


In [137]:
df['TSH'] = df['TSH'].astype(float)


In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        3772 non-null   int32  
 1   sex                        3772 non-null   int32  
 2   on thyroxine               3772 non-null   int32  
 3   query on thyroxine         3772 non-null   int32  
 4   on antithyroid medication  3772 non-null   int32  
 5   sick                       3772 non-null   int32  
 6   pregnant                   3772 non-null   int32  
 7   thyroid surgery            3772 non-null   int32  
 8   I131 treatment             3772 non-null   int32  
 9   query hypothyroid          3772 non-null   int32  
 10  query hyperthyroid         3772 non-null   int32  
 11  lithium                    3772 non-null   int32  
 12  goitre                     3772 non-null   int32  
 13  tumor                      3772 non-null   int32

In [139]:
from sklearn.model_selection import train_test_split

In [140]:
df['on antithyroid medication'].value_counts()

0    3729
1      43
Name: on antithyroid medication, dtype: int64

In [141]:
df['TBG'].value_counts()

0    3772
Name: TBG, dtype: int64

In [142]:
column=['TBG measured','TBG']
df.drop(columns=column,axis=1,inplace=True)

In [143]:
df.columns.tolist()

['age',
 'sex',
 'on thyroxine',
 'query on thyroxine',
 'on antithyroid medication',
 'sick',
 'pregnant',
 'thyroid surgery',
 'I131 treatment',
 'query hypothyroid',
 'query hyperthyroid',
 'lithium',
 'goitre',
 'tumor',
 'hypopituitary',
 'psych',
 'TSH measured',
 'TSH',
 'T3 measured',
 'T3',
 'TT4 measured',
 'TT4',
 'T4U measured',
 'T4U',
 'FTI measured',
 'FTI',
 'referral source',
 'binaryClass']

# creating a model

In [144]:
X1 = df[['age',
 'sex',
 'on thyroxine',
 'query on thyroxine',
 'on antithyroid medication',
 'sick',
 'pregnant',
 'thyroid surgery',
 'I131 treatment',
 'query hypothyroid',
 'query hyperthyroid',
 'lithium',
 'goitre',
 'tumor',
 'hypopituitary',
 'psych',
 'TSH measured',
 'TSH',
 'T3 measured',
 'T3',
 'TT4 measured',
 'TT4',
 'T4U measured',
 'T4U',
 'FTI measured',
 'FTI',
 'referral source']].values
Y1 = df['binaryClass'].values

# Pass 1

In [145]:
X = np.array(X1)  # input features
y = np.array(Y1)  # output labels

# Define the number of partitions
n_splits = 5

# Shuffle the data randomly
np.random.seed(42)
indices = np.random.permutation(len(X))
X = X[indices]
y = y[indices]

# Create the KFold object to generate the partitions
kf = KFold(n_splits=n_splits)

# Train and test the random forest model on each partition
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Partition {i+1}:")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the random forest model on the training set
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Test the random forest model on the test set
    y_pred = model.predict(X_test)
    predicted_integers = [int(round(value)) for value in y_pred]

    print(classification_report(y_test,predicted_integers))

    
    print('-'*25)


Partition 1:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        58
           1       1.00      1.00      1.00       697

    accuracy                           1.00       755
   macro avg       1.00      0.99      1.00       755
weighted avg       1.00      1.00      1.00       755

-------------------------
Partition 2:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        54
           1       1.00      1.00      1.00       701

    accuracy                           1.00       755
   macro avg       1.00      0.99      0.99       755
weighted avg       1.00      1.00      1.00       755

-------------------------
Partition 3:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        66
           1       1.00      1.00      1.00       688

    accuracy                           1.00       754
   macro avg       0.99      0.99    

# Pass 2

In [146]:
# df3 = pd.DataFrame(columns=['Partition','Sub partition','MSE','MAE','RMSE','Accuracy']) # creating an empty Dataframe with some columns


In [147]:
X = np.array(X1)  # input features
y = np.array(Y1)  # output labels

# Define the number of partitions
n_splits = 5

# Shuffle the data randomly
np.random.seed(42)
indices = np.random.permutation(len(X))
# print(indices)
X = X[indices]
y = y[indices]

# Create the KFold object to generate the partitions
kf = KFold(n_splits=n_splits)

# Train and test the model on different partitions
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Partition {i+1}:")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the machine learning model on the training set
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # Test the machine learning model on the remaining partitions
    for j, (train_index2, test_index2) in enumerate(kf.split(X)):
        if j != i:
            X_train2, X_test2 = X[train_index2], X[test_index2]
            y_train2, y_test2 = y[train_index2], y[test_index2]
            
            y_pred = model.predict(X_test2)
            predicted_integers = [int(round(value)) for value in y_pred]
            print(classification_report(y_test2,predicted_integers))      
            print("-"*50)
    print('-'*55)

Partition 1:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        54
           1       1.00      1.00      1.00       701

    accuracy                           1.00       755
   macro avg       1.00      1.00      1.00       755
weighted avg       1.00      1.00      1.00       755

--------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        66
           1       1.00      1.00      1.00       688

    accuracy                           1.00       754
   macro avg       1.00      1.00      1.00       754
weighted avg       1.00      1.00      1.00       754

--------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00       704

    accuracy                           1.00       754
   macro avg 

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        58
           1       1.00      1.00      1.00       697

    accuracy                           1.00       755
   macro avg       1.00      1.00      1.00       755
weighted avg       1.00      1.00      1.00       755

--------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        54
           1       1.00      1.00      1.00       701

    accuracy                           1.00       755
   macro avg       1.00      1.00      1.00       755
weighted avg       1.00      1.00      1.00       755

--------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00       704

    accuracy                           1.00       754
   macro avg       1.00   

In [148]:
df['binaryClass'].value_counts()

1    3481
0     291
Name: binaryClass, dtype: int64

# Pass 1 with confusion matrix

In [149]:
dff = pd.DataFrame(columns=['Partition','Precision','recall','F-score','FPR','Accuracy','Sensitivity','Specificity']) # creating empty data frame with some column

In [150]:
X = np.array(X1)  # input features
y = np.array(Y1)  # output labels

# Define the number of partitions
n_splits = 5

# Shuffle the data randomly
np.random.seed(42)
indices = np.random.permutation(len(X))
X = X[indices]
y = y[indices]

# Create the KFold object to generate the partitions
kf = KFold(n_splits=n_splits)

# Train and test the random forest model on each partition
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Partition {i+1}:")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the random forest model on the training set
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Test the random forest model on the test set
    y_pred = model.predict(X_test)
    predicted_integers = [int(round(value)) for value in y_pred]

    a = confusion_matrix(y_test,predicted_integers)

    precision = a[0][0]/(a[0][0]+a[0][1])
    recall = a[0][0]/(a[0][0]+a[1][0])
    F_score = (2*(precision*recall))/(precision+recall)
    Fal_ps_r = a[0][1]/(a[0][1]+a[1][1])
    Accuracy = (a[0][0]+a[1][1])/(a[0][0]+a[0][1]+a[1][0]+a[1][1])
    Sensitivity = a[0][0]/(a[0][0]+a[1][0])
    Specificity = a[1][1]/(a[1][1]+a[0][1])

    row = [i+1,precision,recall,F_score,Fal_ps_r,Accuracy,Sensitivity,Specificity]
    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"F-score: {F_score}")
    print(f"FPR:{Fal_ps_r} ")
    print(f"Accuracy: {Accuracy}")
    print(f"Sensitivity: {Sensitivity}")
    print(f"Specificity: {Specificity}")

    dff.loc[len(dff)] = row 

    
    print('-'*25)


Partition 1:


precision: 0.9827586206896551
recall: 1.0
F-score: 0.9913043478260869
FPR:0.0014326647564469914 
Accuracy: 0.9986754966887417
Sensitivity: 1.0
Specificity: 0.998567335243553
-------------------------
Partition 2:
precision: 0.9814814814814815
recall: 1.0
F-score: 0.9906542056074767
FPR:0.0014245014245014246 
Accuracy: 0.9986754966887417
Sensitivity: 1.0
Specificity: 0.9985754985754985
-------------------------
Partition 3:
precision: 0.9848484848484849
recall: 0.9848484848484849
F-score: 0.9848484848484849
FPR:0.0014534883720930232 
Accuracy: 0.9973474801061007
Sensitivity: 0.9848484848484849
Specificity: 0.998546511627907
-------------------------
Partition 4:
precision: 1.0
recall: 0.9615384615384616
F-score: 0.9803921568627451
FPR:0.0 
Accuracy: 0.9973474801061007
Sensitivity: 0.9615384615384616
Specificity: 1.0
-------------------------
Partition 5:
precision: 0.9841269841269841
recall: 0.96875
F-score: 0.9763779527559054
FPR:0.0014492753623188406 
Accuracy: 0.9960212201591512
Sens

In [151]:
dff

,Partition,Precision,recall,F-score,FPR,Accuracy,Sensitivity,Specificity
0,1.0,0.982759,1.000000,0.991304,0.001433,0.998675,1.000000,0.998567
1,2.0,0.981481,1.000000,0.990654,0.001425,0.998675,1.000000,0.998575
2,3.0,0.984848,0.984848,0.984848,0.001453,0.997347,0.984848,0.998547
3,4.0,1.000000,0.961538,0.980392,0.000000,0.997347,0.961538,1.000000
4,5.0,0.984127,0.968750,0.976378,0.001449,0.996021,0.968750,0.998551


# Pass 2 with confusion matrix

In [152]:
X = np.array(X1)  # input features
y = np.array(Y1)  # output labels

# Define the number of partitions
n_splits = 5

# Shuffle the data randomly
np.random.seed(42)
indices = np.random.permutation(len(X))
# print(indices)
X = X[indices]
y = y[indices]

# Create the KFold object to generate the partitions
kf = KFold(n_splits=n_splits)

# Train and test the model on different partitions
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Partition {i+1}:")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the machine learning model on the training set
    model = RandomForestRegressor(n_estimators=200, random_state=20)
    model.fit(X_train, y_train)

    # Test the machine learning model on the remaining partitions
    for j, (train_index2, test_index2) in enumerate(kf.split(X)):
        if j != i:
            X_train2, X_test2 = X[train_index2], X[test_index2]
            y_train2, y_test2 = y[train_index2], y[test_index2]
            
            y_pred = model.predict(X_test2)
            predicted_integers = [int(round(value)) for value in y_pred]
            a = confusion_matrix(y_test2,predicted_integers)

            
            precision = a[0][0]/(a[0][0]+a[0][1])
            recall = a[0][0]/(a[0][0]+a[1][0])
            F_score = (2*(precision*recall))/(precision+recall)
            Fal_ps_r = a[0][1]/(a[0][1]+a[1][1])
            Accuracy = (a[0][0]+a[1][1])/(a[0][0]+a[0][1]+a[1][0]+a[1][1])
            Sensitivity = a[0][0]/(a[0][0]+a[1][0])
            Specificity = a[1][1]/(a[1][1]+a[0][1]) 

            print(f"precision: {precision}")
            print(f"recall: {recall}")
            print(f"F-score: {F_score}")
            print(f"FPR:{Fal_ps_r} ")
            print(f"Accuracy: {Accuracy}")
            print(f"Sensitivity: {Sensitivity}")
            print(f"Specificity: {Specificity}")   
            print("-"*50)
    print('-'*55)

Partition 1:


precision: 1.0
recall: 1.0
F-score: 1.0
FPR:0.0 
Accuracy: 1.0
Sensitivity: 1.0
Specificity: 1.0
--------------------------------------------------
precision: 1.0
recall: 1.0
F-score: 1.0
FPR:0.0 
Accuracy: 1.0
Sensitivity: 1.0
Specificity: 1.0
--------------------------------------------------
precision: 1.0
recall: 1.0
F-score: 1.0
FPR:0.0 
Accuracy: 1.0
Sensitivity: 1.0
Specificity: 1.0
--------------------------------------------------
precision: 1.0
recall: 1.0
F-score: 1.0
FPR:0.0 
Accuracy: 1.0
Sensitivity: 1.0
Specificity: 1.0
--------------------------------------------------
-------------------------------------------------------
Partition 2:
precision: 1.0
recall: 1.0
F-score: 1.0
FPR:0.0 
Accuracy: 1.0
Sensitivity: 1.0
Specificity: 1.0
--------------------------------------------------
precision: 1.0
recall: 1.0
F-score: 1.0
FPR:0.0 
Accuracy: 1.0
Sensitivity: 1.0
Specificity: 1.0
--------------------------------------------------
precision: 1.0
recall: 1.0
F-score: 1.0
FPR

# below content is for testing(sample):

In [153]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score,classification_report,confusion_matrix
from sklearn import metrics
import numpy as np

# Load your dataset here
X = np.array(X1)  # input features
y = np.array(Y1)  # output labels

# Define the number of partitions
n_splits = 5

# Shuffle the data randomly
np.random.seed(42)
indices = np.random.permutation(len(X))
X = X[indices]
y = y[indices]

# Create the KFold object to generate the partitions
kf = KFold(n_splits=n_splits)

# Train and test the random forest model on each partition
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Partition {i+1}:")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the random forest model on the training set
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Test the random forest model on the test set
    y_pred = model.predict(X_test)
    predicted_integers = [int(round(value)) for value in y_pred]
    # print('prediction',predicted_integers)
    a= confusion_matrix(y_test,predicted_integers)
    print(classification_report(y_test,predicted_integers))
    print('precision',(a[0][0]/(a[0][0]+a[0][1])))
    print("classification",a)

    print('-'*25)

    # print('test',y_test)
    # print('-'*35)


Partition 1:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        58
           1       1.00      1.00      1.00       697

    accuracy                           1.00       755
   macro avg       1.00      0.99      1.00       755
weighted avg       1.00      1.00      1.00       755

precision 0.9827586206896551
classification [[ 57   1]
 [  0 697]]
-------------------------
Partition 2:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99        54
           1       1.00      1.00      1.00       701

    accuracy                           1.00       755
   macro avg       1.00      0.99      0.99       755
weighted avg       1.00      1.00      1.00       755

precision 0.9814814814814815
classification [[ 53   1]
 [  0 701]]
-------------------------
Partition 3:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        66
           1

# training whole dataset

In [154]:
df

,age,sex,on thyroxine,query on thyroxine,on antithyroid medication,sick,pregnant,thyroid surgery,I131 treatment,query hypothyroid,query hyperthyroid,lithium,goitre,tumor,hypopituitary,psych,TSH measured,TSH,T3 measured,T3,TT4 measured,TT4,T4U measured,T4U,FTI measured,FTI,referral source,binaryClass
0,41,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.300000,1,2.5000,1,125.000000,1,1.140,1,109.000000,1,1
1,23,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4.100000,1,2.0000,1,102.000000,0,0.995,0,110.469649,4,1
2,46,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.980000,0,2.0135,1,109.000000,1,0.910,1,120.000000,4,1
3,70,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.160000,1,1.9000,1,175.000000,0,0.995,0,110.469649,4,1
4,70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.720000,1,1.2000,1,61.000000,1,0.870,1,70.000000,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3767,30,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,5.086766,0,2.0135,0,108.319345,0,0.995,0,110.469649,4,1
3768,68,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.000000,1,2.1000,1,124.000000,1,1.080,1,114.000000,3,1
3769,74,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,5.100000,1,1.8000,1,112.000000,1,1.070,1,105.000000,4,1
3770,72,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.700000,1,2.0000,1,82.000000,1,0.940,1,87.000000,3,1


In [155]:
X_train,X_test,Y_train,Y_test = train_test_split(X1,Y1,test_size=0.2,random_state=0)

In [156]:
from sklearn.ensemble import RandomForestRegressor

In [157]:
rfc = RandomForestRegressor(n_estimators=200)

In [158]:
rfc.fit(X_train,Y_train)

RandomForestRegressor(n_estimators=200)

In [159]:
pred = rfc.predict(X_test)
predicted_integers = [int(round(value)) for value in pred]


In [160]:
print(classification_report(Y_test,predicted_integers))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69
           1       1.00      1.00      1.00       686

    accuracy                           1.00       755
   macro avg       1.00      1.00      1.00       755
weighted avg       1.00      1.00      1.00       755



# Plotting the graph for Pass 1

In [161]:
X1 = df[['age',
 'sex',
 'on thyroxine',
 'query on thyroxine',
 'on antithyroid medication',
 'sick',
 'pregnant',
 'thyroid surgery',
 'I131 treatment',
 'query hypothyroid',
 'query hyperthyroid',
 'lithium',
 'goitre',
 'tumor',
 'hypopituitary',
 'psych',
 'TSH measured',
 'TSH',
 'T3 measured',
 'T3',
 'TT4 measured',
 'TT4',
 'T4U measured',
 'T4U',
 'FTI measured',
 'FTI',
 'referral source']].values
Y1 = df['binaryClass'].values

In [162]:
dff = pd.DataFrame(columns=['Partition','Precision','recall','F-score','FPR','Accuracy','Sensitivity','Specificity']) # creating empty data frame with some column

In [163]:
X = np.array(X1)  # input features
y = np.array(Y1)  # output labels

# Define the number of partitions
n_splits = 5

# Shuffle the data randomly
np.random.seed(42)
indices = np.random.permutation(len(X))
X = X[indices]
y = y[indices]

# Create the KFold object to generate the partitions
kf = KFold(n_splits=n_splits)

# Train and test the random forest model on each partition
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # print(f"Partition {i+1}:")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the random forest model on the training set
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Test the random forest model on the test set
    y_pred = model.predict(X_test)
    predicted_integers = [int(round(value)) for value in y_pred]

    a = confusion_matrix(y_test,predicted_integers)

    precision = a[0][0]/(a[0][0]+a[0][1])
    recall = a[0][0]/(a[0][0]+a[1][0])
    F_score = (2*(precision*recall))/(precision+recall)
    Fal_ps_r = a[0][1]/(a[0][1]+a[1][1])
    Accuracy = (a[0][0]+a[1][1])/(a[0][0]+a[0][1]+a[1][0]+a[1][1])
    Sensitivity = a[0][0]/(a[0][0]+a[1][0])
    Specificity = a[1][1]/(a[1][1]+a[0][1])

    row = [i+1,precision,recall,F_score,Fal_ps_r,Accuracy,Sensitivity,Specificity]
    # print(f"precision: {precision}")
    # print(f"recall: {recall}")
    # print(f"F-score: {F_score}")
    # print(f"FPR:{Fal_ps_r} ")
    # print(f"Accuracy: {Accuracy}")
    # print(f"Sensitivity: {Sensitivity}")
    # print(f"Specificity: {Specificity}")

    dff.loc[len(dff)] = row 

    
    # print('-'*25)


In [164]:
dff

,Partition,Precision,recall,F-score,FPR,Accuracy,Sensitivity,Specificity
0,1.0,0.982759,1.000000,0.991304,0.001433,0.998675,1.000000,0.998567
1,2.0,0.981481,1.000000,0.990654,0.001425,0.998675,1.000000,0.998575
2,3.0,0.984848,0.984848,0.984848,0.001453,0.997347,0.984848,0.998547
3,4.0,1.000000,0.961538,0.980392,0.000000,0.997347,0.961538,1.000000
4,5.0,0.984127,0.968750,0.976378,0.001449,0.996021,0.968750,0.998551


# Visualization

In [165]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

In [166]:
fig1 = dff.iplot(kind='bar',x='Partition',y='Precision',title='Precision in Partitions: Pass 1',xTitle='Partitions',yTitle='Precision Percentage',asFigure=True)
fig1

In [167]:
fig2 = dff.iplot(kind='bar',x='Partition',y='Precision',title='Evaluations in Each Partitions',xTitle='Partitions',yTitle='Evaluation Percentage',categories="Partition",asFigure=True)
fig2

In [168]:
fig3 = px.scatter_3d(dff,
        x='Partition', y='Precision', z='Accuracy',
        color="Partition")
fig3

In [169]:
fig4 = px.scatter_3d(dff,
        x='Partition', y='recall', z='Accuracy',
        color="Partition")
fig4

In [170]:
fig5 = px.scatter_3d(dff,
        x='Partition', y='F-score', z='Partition',
        color="Partition")
fig5

In [175]:
fig6 = px.scatter_3d(dff,
        x='Partition', y='Sensitivity', z='Specificity',
        color="Partition")
fig6

# plotting graph for Pass 2

In [171]:
df3 = pd.DataFrame(columns=['Partition','Sub partition','Precision','recall','F-score','FPR','Accuracy','Sensitivity','Specificity']) # creating an empty Dataframe with some columns

In [172]:
X = np.array(X1)  # input features
y = np.array(Y1)  # output labels

# Define the number of partitions
n_splits = 5

# Shuffle the data randomly
np.random.seed(42)
indices = np.random.permutation(len(X))
# print(indices)
X = X[indices]
y = y[indices]

# Create the KFold object to generate the partitions
kf = KFold(n_splits=n_splits)

# Train and test the model on different partitions
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # print(f"Partition {i+1}:")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the machine learning model on the training set
    model = RandomForestRegressor(n_estimators=200, random_state=20)
    model.fit(X_train, y_train)

    # Test the machine learning model on the remaining partitions
    for j, (train_index2, test_index2) in enumerate(kf.split(X)):
        if j != i:
            X_train2, X_test2 = X[train_index2], X[test_index2]
            y_train2, y_test2 = y[train_index2], y[test_index2]
            
            y_pred = model.predict(X_test2)
            predicted_integers = [int(round(value)) for value in y_pred]
            a = confusion_matrix(y_test2,predicted_integers)

            
            precision = a[0][0]/(a[0][0]+a[0][1])
            recall = a[0][0]/(a[0][0]+a[1][0])
            F_score = (2*(precision*recall))/(precision+recall)
            Fal_ps_r = a[0][1]/(a[0][1]+a[1][1])
            Accuracy = (a[0][0]+a[1][1])/(a[0][0]+a[0][1]+a[1][0]+a[1][1])
            Sensitivity = a[0][0]/(a[0][0]+a[1][0])
            Specificity = a[1][1]/(a[1][1]+a[0][1]) 

            # print(f"precision: {precision}")
            # print(f"recall: {recall}")
            # print(f"F-score: {F_score}")
            # print(f"FPR:{Fal_ps_r} ")
            # print(f"Accuracy: {Accuracy}")
            # print(f"Sensitivity: {Sensitivity}")
            # print(f"Specificity: {Specificity}") 
            row = [i+1,j+1,precision,recall,F_score,Fal_ps_r,Accuracy,Sensitivity,Specificity]
            df3.loc[len(df3)] = row # inserting evaluation data in a dataframe
    #         print("-"*50)
    # print('-'*55)

In [173]:
df3

,Partition,Sub partition,Precision,recall,F-score,FPR,Accuracy,Sensitivity,Specificity
0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
1,1.0,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
2,1.0,4.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
3,1.0,5.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
4,2.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
5,2.0,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
6,2.0,4.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
7,2.0,5.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
8,3.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
9,3.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [182]:
fig7 = df3.iplot(kind='bubble',x='Partition',y='Accuracy',size='Sub partition',xTitle='Partitions',
                 yTitle='Evaluation Percentage',categories="Sub partition",title="Accuracy in Each Partion: Pass 2",asFigure=True)
fig7

In [183]:
fig8 = df3.iplot(kind='bubble',x='Partition',y='Precision',size='Sub partition',xTitle='Partitions',
                 yTitle='Evaluation Percentage',categories="Sub partition",title="Precision in Each Partion: Pass 2",asFigure=True)
fig8

In [184]:
fig9 = df3.iplot(kind='bubble',x='Partition',y='FPR',size='Sub partition',xTitle='Partitions',
                 yTitle='Evaluation Percentage',categories="Sub partition",title="False Positive rate in Each Partion: Pass 2",asFigure=True)
fig9

In [185]:
fig10 = dff.iplot(kind='surface', x='Partition',y='Accuracy',color='rainbow',asFigure=True)
fig10

In [196]:
fig11 = df3.iplot(kind='scatter3d', x='Partition',y='Accuracy',z='Sub partition',asFigure=True,xTitle='partitions',yTitle='Accuracy in partitions',zTitle='Sub partitions')
fig11

In [199]:
fig12 = df3.iplot(kind='scatter3d', x='Partition',y='FPR',z='Sub partition',asFigure=True,xTitle='partitions',yTitle='False positive rate in partitions',zTitle='Sub partitions',colors=["dodgerblue", "lime", "tomato"])
fig12